### In this assignment, you will build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page.

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.




#### NASA Mars News
Scrape the NASA Mars News Site (https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
import re
import pandas as pd


mars_docs = {}


##### Mac users

In [27]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

##### Windows Users

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [29]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Retrieve first instance (not all (find_all)) of element that contains article text.
top_headline = soup.find('div', class_='list_text')
news_title = soup.find('div', class_='list_text').find('div', class_='content_title').get_text()
news_p = soup.find('div', class_='list_text').find('div', class_='article_teaser_body').get_text()
print('Top Mars headline is "{}: {}"'.format(news_title, news_p))

mars_docs["news_title"] = news_title
mars_docs["news_p"] = news_p

Top Mars headline is "Update on Opportunity Rover after Martian Dust Storm: One month since increasing their commanding frequency, engineers have yet to hear from NASA's Opportunity rover."


#### JPL Mars Space Images - Featured Image
Visit the url for JPL Featured Space Image here (https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [30]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# navigate to view featured image
browser.click_link_by_partial_text('FULL IMAGE')

# note the above page sometimes takes a long time to load and the 'more info' button isnt available
time.sleep(15)
browser.click_link_by_partial_text('more info')



In [31]:
# Scrape the browser into soup and use soup to find the full resolution image of mars
# Save the image url to a variable called `img_url`

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# build out full url for image
image_partial_url = soup.find("img", class_="main_image")["src"]
featured_image_url = "https://www.jpl.nasa.gov"+image_partial_url
print(featured_image_url)

mars_docs["featured_image_url"] = featured_image_url


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA03519_hires.jpg


#### Mars Weather
Visit the Mars Weather twitter account here (https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. 

Save the tweet text for the weather report as a variable called mars_weather.

In [32]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(15)

In [33]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

mars_weather=soup.find(string=re.compile("Sol"))
print(mars_weather) 

mars_docs["mars_weather"] = mars_weather

Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59


#### Mars Facts
Visit the Mars Facts webpage here (http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string.

In [3]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [36]:
# import pandas as pd

facts_table = pd.read_html(url)
facts_df = facts_table[0]
facts_df.columns = ['Description', 'Value']
facts_df.set_index("Description", inplace= True)
facts_html = facts_df.to_html()
facts_html

mars_docs["facts_html"] = facts_html
facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    

#### Mars Hemispheres
Visit the USGS Astrogeology site here (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [37]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [38]:
hemisphere_image_urls = []
urls = []

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

items = soup.find_all('div', class_="description")

for item in items:
    print(item.a['href'])
    urls.append(item.a['href'])

urls

/search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced


['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [40]:
# items loop
for i in urls:
    url = 'https://astrogeology.usgs.gov' + i
    browser.visit(url)
    
    # get new soup html
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # store in dict hemisphere name and image URL
    dict = {}
    title = soup.find('div', class_="content").find('h2').text
    partial_url = soup.find('img', class_="wide-image")["src"]
    img_url = 'https://astrogeology.usgs.gov' + partial_url
    dict["title"]= title
    dict["img_url"]= img_url
    hemisphere_image_urls.append(dict)

print(hemisphere_image_urls)

mars_docs["hemisphere_image_urls"] = hemisphere_image_urls


In [41]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [42]:
mars_docs

{'news_title': 'Update on Opportunity Rover after Martian Dust Storm',
 'news_p': "One month since increasing their commanding frequency, engineers have yet to hear from NASA's Opportunity rover.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA03519_hires.jpg',
 'mars_weather': 'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59',
 'facts_html': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n   